In [26]:
import numpy as np

Supongamos que tenemos una matriz NumPy que representa algunos datos de un experimento:

In [27]:
temperature_0 = np.random.random(1024)

In [28]:
temperature_from_station_20=np.random.random(1024)

In [29]:
np.info(temperature_0)

class:  ndarray
shape:  (1024,)
strides:  (8,)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x5575ed6ec360
byteorder:  little
byteswap:  False
type: float64


Imaginemos también que estos puntos de datos se registraron desde una estación meteorológica que muestreó la temperatura, por ejemplo, cada 10 segundos. 

In [30]:
dt = 10.0


La adquisición de datos comenzó en un momento particular,
que también tendremos que registrar. 

In [31]:
start_time = 1375204299 

Y, por supuesto, tenemos que saber que los datos provienen
de la Estación Meteorológica 15:


In [32]:
station = 15

In [33]:
np.savez('weather.npz', data=temperature_0, start_time=start_time, station=station)

Podemos recuperar los valores del archivo con np.load:

In [34]:
out = np.load('weather.npz')

imprimimos "out"

In [35]:
out['data']

array([0.57254658, 0.4899147 , 0.00867605, ..., 0.25405927, 0.52466526,
       0.43856955])

NpzFile (fid)

Un objeto tipo diccionario con carga lenta de archivos en el archivo comprimido
archivo proporcionado en la construcción.

`NpzFile` se usa para cargar archivos en el archivo de datos NumPy` `.npz``
formato. Se supone que los archivos en el archivo tienen una extensión `` .npy``,
otros archivos son ignorados

Las matrices y las cadenas de archivos se cargan perezosamente en cualquiera
obtener acceso a elementos usando `` obj ['clave'] `` o búsqueda de atributos usando
`` obj.f.key ''. Una lista de todos los archivos (sin extensiones `` .npy``) puede
se obtendrá con `` obj.files '' y el objeto ZipFile en sí mismo usando
`` obj.zip ''.

Atributos
----------
archivos: lista de str
    Lista de todos los archivos en el archivo con una extensión `` .npy``.
zip: instancia de ZipFile
    El objeto ZipFile se inicializó con el archivo comprimido.
f: instancia de BagObj
    Un objeto en el que el atributo se puede realizar como alternativa
    para obtener acceso al elemento en la propia instancia de `NpzFile`.
allow_pickle: bool, opcional
    Permitir cargar datos encurtidos. Valor predeterminado: falso

    .. versionchanged :: 1.16.3
        Falso predeterminado en respuesta a CVE-2019-6446.

pickle_kwargs: dict, opcional
    Argumentos de palabras clave adicionales para pasar a pickle.load.
    Estos solo son útiles al cargar matrices de objetos guardadas en
    Python 2 cuando se usa Python 3.

Parámetros
----------
fid: archivo o str
    El archivo comprimido para abrir. Este es un objeto similar a un archivo
    o una cadena que contiene la ruta al archivo.
own_fid: bool, opcional
    Si NpzFile debería cerrar el identificador de archivo.
    Requiere que `fid` sea un objeto similar a un archivo.

Ejemplos
--------
>>> desde la importación de archivos temporales TemporaryFile
>>> archivo de salida = TemporaryFile ()
>>> x = np. naranja (10)
>>> y = np.sin (x)
>>> np.savez (outfile, x = x, y = y)
>>> _ = outfile.seek (0)

>>> npz = np.load (archivo de salida)
>>> isinstance (npz, np.lib.io.NpzFile)
Cierto
>>> ordenado (npz.files)
['x', 'y']
>>> npz ['x'] # getitem acceso
matriz ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
>>> npz.fx # búsqueda de atributos
matriz ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [36]:
out['start_time']

array(1375204299)

In [37]:
np.info(out['start_time'])

class:  ndarray
shape:  ()
strides:  ()
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x5575ed6a8170
byteorder:  little
byteswap:  False
type: int64


In [38]:
out['station']

array(15)

In [39]:
np.info(out['station'])

class:  ndarray
shape:  ()
strides:  ()
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x5575ed6e0f90
byteorder:  little
byteswap:  False
type: int64


Hasta aquí todo bien.
Pero, ¿qué pasa si tenemos más de una cantidad por estación? 

¿Digamos que también hay datos de velocidad del viento para registrar?



In [40]:
wind = np.random.random(2048)

In [41]:
dt_wind = 5.0   # Wind sampled every 5 secondsÇ

In [42]:
import h5py

In [43]:
f = h5py.File('weather.hdf5')

<ipython-input-43-bc60a7513f3d>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File('weather.hdf5')


In [44]:
 f['/15/temperature'] = temperature_0

In [45]:
f['/15/temperature'].attrs['dt'] = 10.0

In [46]:
f['/15/temperature'].attrs['start_time'] = 1375204299

In [47]:
f['/15/wind'] = wind

In [48]:
f['/15/wind'].attrs['dt'] = 5.0

In [49]:
f['/20/temperature'] = temperature_from_station_20

Este ejemplo ilustra dos de las "características asesinas" de HDF5:
organización en grupos y atributos jerárquicos. 
Grupos, como carpetasen un sistema de archivos, le permite almacenar conjuntos de datos relacionados juntos. 
En este caso, las mediciones de temperatura y
viento de la misma estación meteorológica se almacenan juntas en grupos llamados "/ 15", "/ 20", etc. 
Los atributos le permiten adjuntar metadatos descriptivos directamente a los datos que describe .
Entonces, si le entrega este archivo a un colega, 
ella puede descubrir fácilmente la información necesaria para dar sentido a los datos:

In [51]:
dataset = f['/15/temperature']

In [52]:
for key, value in dataset.attrs.items():
    print ('%s: %s' % (key, value))

dt: 10.0
start_time: 1375204299


Lidiando con grandes volúmenes de datos
Como un "pegamento" de alto nivel lenguaje, Python se usa cada vez más para la visualización rápida de grandes conjuntos de datos y para coordinar cálculos a gran escala que se ejecutan en lenguajes compilados como C y FORTRAN. Ahora es relativamente común tratar con conjuntos de datos de cientos de gigabytes o incluso terabytes de tamaño; HDF5 en sí puede escalar a exabytes.

En todo menos en el más grande máquinas, no es factible cargar dichos conjuntos de datos directamente en la memoria. Una de las mayores fortalezas de HDF5 es su soporte para subconjuntos y E / S parciales. Por ejemplo, tomemos el conjunto de datos de "temperatura" de 1024 elementos que creamos anteriormente:

In [53]:
dataset = f['/15/temperature']

In [54]:
dataset[0:10]

array([0.57254658, 0.4899147 , 0.00867605, 0.20335622, 0.50485746,
       0.79251988, 0.63135178, 0.76426047, 0.88351243, 0.28527645])

In [55]:
dataset[0:10:2]

array([0.57254658, 0.00867605, 0.50485746, 0.63135178, 0.88351243])

Tenga en cuenta que los datos reales viven en el disco; Cuando el corte se aplica a un conjunto de datos HDF5, los datos apropiados se encuentran y se cargan en la memoria. Rebanar de esta manera aprovecha las capacidades de subconjunto subyacentes de HDF5 y, en consecuencia, es muy rápido.

Otra gran cosa acerca de HDF5 es que tiene control sobre cómo el almacenamiento esta asignado. Por ejemplo, a excepción de algunos metadatos, un nuevo conjunto de datos ocupa cero espacio y, por defecto, los bytes solo se usan en el disco para contener los datos que realmente escribe.

Por ejemplo, aquí hay un conjunto de datos de 2 terabytes que puede crear en casi cualquier computadora:



In [56]:
big_dataset = f.create_dataset('big', shape=(1024, 1024, 1024, 512), dtype='float32')


Aunque todavía no se ha asignado almacenamiento, todo el "espacio" del conjunto de datos está disponible para nosotros. Podemos escribir en cualquier parte del conjunto de datos, y solo se utilizan los bytes necesarios en el disco para guardar los datos:



In [57]:
big_dataset[344, 678, 23, 36] = 42.0


Cuando el almacenamiento es escaso, incluso puede utilizar una compresión transparente conjunto de datos por conjunto de datos (consulte el Capítulo 4 ):

In [58]:
compressed_dataset = f.create_dataset('comp', shape=(1024,), dtype='int32', compression='gzip')

In [59]:
compressed_dataset[:] = np.arange(1024)


In [60]:
compressed_dataset[:]

array([   0,    1,    2, ..., 1021, 1022, 1023], dtype=int32)

¿Qué es exactamente HDF5?
HDF5 es un Gran mecanismo para almacenar grandes matrices numéricas de tipo homogéneo , para modelos de datos que pueden organizarse jerárquicamente y beneficiarse del etiquetado de conjuntos de datos con metadatos arbitrarios .



Es bastante diferentede bases de datos relacionales de estilo SQL. HDF5 tiene bastantes trucos organizativos bajo la manga (consulte el Capítulo 8 , por ejemplo), pero si necesita forzar relaciones entre valores en varias tablas, o si desea realizar uniones en sus datos, una base de datos relacional probablemente sea más apropiada . Del mismo modo, para pequeños conjuntos de datos 1D, debe poder leer en máquinas sin HDF5 instalado. Los formatos de texto como CSV (con todas sus verrugas) son una alternativa razonable.

HDF5 es casi perfecto si hace un uso mínimo de las funciones relacionales y necesita un rendimiento muy alto, E / S parcial, organización jerárquica y metadatos arbitrarios.

Entonces, ¿qué es, específicamente, "HDF5"? Yo diría que consta de tres cosas:

Una especificación de archivo y modelo de datos asociado.
Una biblioteca estándar con acceso a API disponible desde C, C ++, Java, Python y otros.
Un ecosistema de software, que consta de programas cliente que utilizan HDF5 y "plataformas de análisis" como MATLAB, IDL y Python.
HDF5: el archivo


En los breves ejemplos anteriores, vio los tres elementos principales del modelo de datos HDF5: conjuntos de datos , objetos en forma de matriz que almacenan sus datos numéricos en el disco; grupos , contenedores jerárquicos que almacenan conjuntos de datos y otros grupos; y atributos , bits de metadatos definidos por el usuario que se pueden adjuntar a conjuntos de datos (¡y grupos!).

Usando estas abstracciones básicas, los usuarios pueden construir "formatos de aplicación" específicos que organizan los datos en un método apropiado para el dominio del problema. Por ejemplo, nuestro código de "estación meteorológica" utilizó un grupo para cada estación y conjuntos de datos separados para cada parámetro medido, con atributos para contener información adicional sobre lo que significan los conjuntos de datos. Es muy común que los laboratorios u otras organizaciones acuerden un "formato dentro de un formato" que especifique qué disposición de grupos, conjuntos de datos y atributos se utilizará para almacenar información.

Dado que HDF5 se encarga de todos los problemas multiplataforma Al igual que la resistencia, compartir datos con otros grupos se convierte en una simple cuestión de manipular grupos, conjuntos de datos y atributos para obtener el resultado deseado. Y porque los archivosse autodescriben , incluso conocer el formato de la aplicación no suele ser necesario para extraer datos del archivo. Simplemente puede abrir el archivo y explorar su contenido:

In [61]:
f.keys()

<KeysViewHDF5 ['15', '20', 'big', 'comp']>

In [62]:
f['/15'].keys()

<KeysViewHDF5 ['temperature', 'wind']>

Cualquiera que haya pasado horas jugueteando con los desplazamientos de bytes mientras intenta leer formatos binarios "simples" puede apreciar esto.

Finalmente, el diseño de bytes de bajo nivel de un archivo HDF5 en disco es una especificación abierta. No hay misterios sobre cómo funciona, en contraste con los formatos binarios patentados. Y aunque las personas suelen utilizar la biblioteca proporcionada por el Grupo HDF para acceder a los archivos, nada le impide escribir su propio lector si lo desea.

HDF5: la biblioteca
La especificación de archivos HDF5 y la biblioteca de código abierto son mantenidas por HDF Group , una organización sin fines de lucro con sede en Champaign, Illinois. Anteriormente parte de la Universidad de Illinois Urbana-Champaign, el producto principal del Grupo HDF es la biblioteca de software HDF5.

Escrita en C, con enlaces adicionales para C ++ y Java, esta biblioteca es lo que la gente suele decir cuando dice "HDF5". Las dos interfaces Python más populares, PyTables y h5py, están diseñadas para usar la bibl

populares, PyTables y h5py, están diseñadas para usar la biblioteca C proporcionada por el Grupo HDF .

Un punto importante es que esta biblioteca se mantiene activamente, y los desarrolladores ponen un fuerte énfasis en la compatibilidad con versiones anteriores. Esto se aplica tanto a los archivos que produce la biblioteca como a los programas que usan la API. La compatibilidad de archivos es imprescindible para un formato de archivo como HDF5. Esta atención cuidadosa a la compatibilidad API es la razón principal por la que los paquetes como h5py y PyTables han podido obtener tracción con muchas versiones diferentes de HDF5 instaladas en la naturaleza.

Debe tener confianza al usar HDF5 para el almacenamiento de datos científicos, incluido el almacenamiento a largo plazo. Y dado que tanto la biblioteca como el formato son de código abierto, sus archivos serán legibles incluso si un meteorito destruye Illinois.

HDF5: el ecosistema
Finalmente, un aspectolo que hace que HDF5 sea particularmente útil es que puede leer y escribir archivos desde casi todas las plataformas. El lenguaje IDL ha soportado HDF5 durante años; MATLAB tiene un soporte similar y ahora incluso utiliza HDF5 como formato predeterminado para sus archivos de guardado ".mat". Los enlaces también están disponibles para Python, C ++, Java, .NET y LabView, entre otros. Los usuarios institucionales incluyen el Sistema de Observación de la Tierra de la NASA, cuyo formato "EOS5" es un formato de aplicación en la parte superior del contenedor HDF5, como en el ejemplo mucho más simple anterior. Incluso la versión más reciente del formato NetCDF de la competencia, NetCDF4, se implementa utilizando grupos, conjuntos de datos y atributos HDF5.

Espero haber podido compartir con ustedes algunas de las cosas que hacen que HDF5 sea tan emocionante para el uso científico. A continuación, revisaremos los conceptos básicos de cómo funciona HDF5 y comenzaremos a usarlo desde Python.

Capítulo 2. Comenzando
Conceptos básicos de HDF5
Antes de pasar a los ejemplos de código de Python, es útil tomarse unos minutos para abordar cómo es HDF5 en sí organizado. La Figura 2-1 muestra una caricatura de las diversas capas lógicas involucradas cuando se usa HDF5. Las capas sombreadas en azul son internas de la propia biblioteca; Las capas en verde representan software que utiliza HDF5.

La mayoría del código del cliente, incluido el Los paquetes de Python h5py y PyTables, utilizan la API nativa de C (HDF5 está escrito en C). Como vimos en la introducción, el modelo de datos HDF5 consta de tres abstracciones públicas principales : conjuntos de datos (ver Capítulo 3 ), grupos (ver Capítulo 5 ) y atributos (ver Capítulo 6 ) además de un sistema para representar tipos. La API de C (y el código de Python encima) está diseñada para manipular estos objetos.

HDF5 usa una variedadde estructuras de datos internos para representar grupos, conjuntos de datos y atributos. Por ejemplo, los grupos tienen sus entradas indexadas usando estructuras llamadas "árboles B", que hacen que la recuperación y la creación de miembros del grupo sean muy rápidas, incluso cuando se almacenan cientos de miles de objetos en un grupo (consulte Cómo se almacenan realmente los grupos ). En general, solo se preocupará por estas estructuras de datos cuando se trata de consideraciones de rendimiento. Por ejemplo, cuando se usa almacenamiento fragmentado (consulte el Capítulo 4 ), es importante comprender cómo se organizan los datos en el disco.

Los dos siguientes las capas tienen que ver con cómo sus datos llegan al disco. Todos los objetos HDF5 viven en un espacio de direcciones lógicas 1D, como en un archivo normal. Sin embargo, hay una capa adicional entre este espacio y la disposición real de bytes en el disco. Los controladores HDF5 se encargan de la mecánica de escribir en el disco, y en el proceso pueden hacer algunas cosas increíbles.




Figura 2-1. La biblioteca HDF5: el azul representa componentes dentro de la biblioteca HDF5; el verde representa el código del "cliente" que llama a HDF5; el gris representa los recursos provistos por el sistema operativo.
Por ejemplo, el corecontrolador HDF5le permite usar archivos que viven completamente en la memoria y son increíblemente rápidos. El familyconductorle permite dividir un solo archivo en piezas de tamaño regular. Y el mpiocontrolador le permite acceder al mismo archivo desde múltiples procesos paralelos, utilizando la biblioteca de interfaz de paso de mensajes (MPI) ( MPI y Parallel HDF5 ). Todo esto es transparente para el código que funciona en el nivel superior de grupos, conjuntos de datos y atributos.

Configurando
Eso es suficiente material de fondo por ahora. ¡Comencemos con Python! ¿Pero qué Python?

Python 2 o Python 3?
Se está produciendo un gran cambio en la comunidad de Python. Con los años, Python ha acumulado una serie de características y errores que se han considerado indeseables. Los ejemplos van desde paquetes que usan convenciones de nomenclatura inconsistentes hasta deficiencias en la forma en que se manejan las cadenas. Para abordar estos problemas, se decidió lanzar una nueva versión principal (Python 3) que se liberaría del "bagaje" de las viejas decisiones en la línea Python 2.

Python 2.7, la versión menor más reciente de la serie Python, también será la última versión 2.X. Aunque se actualizará con correcciones de errores durante un período prolongado de tiempo, el nuevo desarrollo de código Python ahora se lleva a cabo exclusivamente en la línea 3.X. El NumPypackage, h5py, PyTables y muchos otros paquetes ahora son compatibles con Python 3. Si bien (en mi opinión) es un poco pronto para recomendar que los recién llegados comiencen con Python 3, el futuro está claro.

Por el momento, hay dos versiones principales de Python ampliamente implementadas simultáneamente. Como la mayoría de las personas en la comunidad Python están acostumbradas a Python 2, los ejemplos en este libro también están escritos para Python 2. En su mayor parte, las diferencias son menores; por ejemplo, en Python 3, la sintaxis para printes print(foo), en lugar de print foo. Donde sea probable que ocurran incompatibilidades (principalmente con los tipos de cadena y ciertos métodos de estilo de diccionario), estos se anotarán en el texto.

Código de "portabilidad" a Python 3 no es realmente tan difícil; después de todo, sigue siendo Python. Algunas de las características más valiosas en Python 3 

Por el momento, hay dos versiones principales de Python ampliamente implementadas simultáneamente. Como la mayoría de las personas en la comunidad Python están acostumbradas a Python 2, los ejemplos en este libro también están escritos para Python 2. En su mayor parte, las diferencias son menores; por ejemplo, en Python 3, la sintaxis para printes print(foo), en lugar de print foo. Donde sea probable que ocurran incompatibilidades (principalmente con los tipos de cadena y ciertos métodos de estilo de diccionario), estos se anotarán en el texto.

Código de "portabilidad" a Python 3 no es realmente tan difícil; después de todo, sigue siendo Python. Algunas de las características más valiosas en Python 3 ya están presentes en Python 2.7. También está disponible una herramienta gratuita ( 2to3) que puede realizar la mayoría de los cambios mecánicos, por ejemplo, cambiar las printdeclaraciones a print()llamadas a funciones. Consulte la guía de migración (y la 2to3herramienta) en http://www.python.org .

Ejemplos de código
Para empezar, la mayoría de los ejemplos de código seguirán este formulario:

>>> a = 1.0
>>> print a
1.0
O, dado que Python imprime objetos por defecto, una forma aún más simple:

>>> a
1.0
Las líneas que comienzan con >>>representan la entrada a Python ( >>>siendo el indicador de Python predeterminado); otras líneas representan salida. Algunos de los ejemplos más largos, donde el resultado del programa no se muestra, omiten el >>>en aras de la claridad.

Los ejemplos destinados a ejecutarse desde la línea de comandos comenzarán con el $indicador ' ' de estilo Unix :

$ python --version
Python 2.7.3
Finalmente, para evitar abarrotar los ejemplos, la mayoría de los fragmentos de código que encontrará aquí supondrán que se han importado los siguientes paquetes:

>>> import numpy as np      # Provides array object and data type objects
>>> import h5py             # Provides access to HDF5
NumPy
"NumPy" es el paquete estándar de matriz numérica en el mundo de Python. Este libro asume que tienes algo de experiencia con NumPy (y, por supuesto, con Python), incluido cómo usar objetos de matriz.

Incluso si ha usado NumPy anteriormente, vale la pena revisar algunos datos básicos sobre cómo funcionan las matrices. Primero, todos los arreglos NumPy tienen datos fijostipo o "dtype", representado por los objetos dtype . Por ejemplo, creemos una matriz de enteros simple de 10 elementos

>>> arr = np.arange(10)
>>> arr
array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
El tipo de datos de esta matriz viene dado por arr.dtype:

>>> arr.dtype
dtype('int32')
Estos objetos dtype son cómo NumPy comunica información de bajo nivel sobre el tipo de datos en la memoria. Para el caso de nuestra matriz de 10 elementos de enteros de 4 bytes (32 bits o "int32" en la jerga NumPy), hay un trozo de memoria en algún lugar de 40 bytes que contiene los valores del 0 al 9. Otro código que recibe el arrEl objeto puede inspeccionar el objeto dtype para descubrir qué representa el contenido de la memoria.

NOTA
El ejemplo anterior podría imprimirse dtype('int64')en su sistema. Todo esto significa que el tamaño entero predeterminado disponible para Python es de 64 bits, en lugar de 32 bits.

HDF5 utiliza un muy sistema de tipo similar; cada "matriz" o conjunto de datos en un archivo HDF5 tiene un tipo fijo representado por un objeto de tipo. El h5pypaquete mapea automáticamente el sistema de tipo HDF5 en los tipos NumPy, lo que, entre otras cosas, facilita el intercambio de datos con NumPy. El Capítulo 3 entra en más detalles sobre este proceso.

Rebanar esOtra característica central de NumPy. Esto significa acceder a porciones de una matriz NumPy. Por ejemplo, para extraer los primeros cuatro elementos de nuestra matriz arr:

>>> out = arr[0:4]
>>> out
array([0, 1, 2, 3])
También puedes especificar un "paso" o pasos entre puntos en el segmento:

>>> out = arr[0:4:2]
>>> out
array([0, 2])
Cuando hablemos con HDF5, tomaremos prestada esta sintaxis de "corte" para permitir cargar solo partes de un conjunto de datos.

En el mundo NumPy, la división se implementa de una manera inteligente, lo que generalmente crea matrices que se refieren a los datos en la matriz origin

en lugar de copias independientes. Por ejemplo, el outobjeto anterior es probablemente una "vista" en la arrmatriz original . Podemos probar esto:

>>> out[:] = 42
>>> out
array([42, 42])
>>> arr
array([42,  1, 42,  3,  4,  5,  6,  7,  8,  9])
Esto significa que rebanar es generalmente una operación muy rápida. Pero debe tener cuidado de crear explícitamente una copia si desea modificar la matriz resultante sin que sus cambios vuelvan al original:

>>> out2 = arr[0:4:2].copy()
PRECAUCIÓN
Olvidar hacer una copia antes de modificar un "segmento" de la matriz es un error muy común, especialmente para las personas acostumbradas a entornos como IDL. Si eres nuevo en NumPy, ¡ten cuidado!

Como veremos más adelante, afortunadamente esto no se aplica a las porciones leídas de los conjuntos de datos HDF5. Cuando lea el archivo, ya que los datos están en el disco, siempre obtendrá una copia.

HDF5 y h5py
Utilizaremos el paquete "h5py" para hablar con HDF5. Este paquete contieneenvoltorios de alto nivel para los objetos HDF5 de archivos, grupos, conjuntos de datos y atributos, así como extensivos envoltorios de bajo nivel para las estructuras y funciones de datos HDF5 C. Los ejemplos en este libro asumen h5py versión 2.2.0 o posterior, que puede obtener en http://www.h5py.org .

Debe tener en cuenta que h5py no es el único paquete comúnmente utilizado para hablar con HDF5. PyTables es un paquete de base de datos científica basado en HDF5 que agrega indexación de conjuntos de datos y un sistema de tipos adicional. Como hablaremos de construcciones HDF5 nativas, nos atendremos a h5py, pero le recomiendo que también consulte PyTables si esas características le interesan.

Si estás en Linux, generalmente es una buena idea instalar la biblioteca HDF5 a través de su administrador de paquetes. En Windows, puede descargar un instalador desde http://www.h5py.org , o usar una de las muchas distribucione

de Python que incluyen HDF5 / h5py, como PythonXY, Anaconda de Continuum Analytics o Enthought Python Distribution.

IPython
Además de NumPy y h5py / HDF5, IPython es un componente imprescindible si va a realizar un análisis o desarrollo exhaustivo con Python. En su forma más básica, IPython es un intérprete de intérprete de reemplazo que agrega características como el historial de comandos y la finalización de tabulación para los atributos del objeto. También tiene toneladas de características adicionales para procesamiento paralelo, visualización de gráficos tipo "cuaderno" de estilo MATLAB, y más.

La mejor manera de explorar las características de este libro es abrir un indicador de IPython, siguiendo los ejemplos. La finalización de tabulación solo vale la pena, ya que le permite ver rápidamente los atributos de módulos y objetos. El paquete h5py está específicamente diseñado para ser "explorable" en este sentido. Por ejemplo, si desea descubrir qué propiedades y métodos existen en el Fileobjeto (vea Su primer archivo HDF5 ), escriba h5py.File.y golpee la tecla Tab:

>>> h5py.File.<TAB>
h5py.File.attrs           h5py.File.get             h5py.File.name
h5py.File.close           h5py.File.id              h5py.File.parent
h5py.File.copy            h5py.File.items           h5py.File.ref
h5py.File.create_dataset  h5py.File.iteritems       h5py.File.require_dataset
h5py.File.create_group    h5py.File.iterkeys        h5py.File.require_group
h5py.File.driver          h5py.File.itervalues      h5py.File.userblock_size
h5py.File.fid             h5py.File.keys            h5py.File.values
h5py.File.file            h5py.File.libver          h5py.File.visit
h5py.File.filename        h5py.File.mode            h5py.File.visititems
h5py.File.flush           h5py.File.mro
Para obtener más información sobre una propiedad o método, use ?después de su nombre:

>>> h5py.File.close?
Type:       instancemethod
Base Class: <type 'instancemethod'>
String Form:<unbound method File.close>
Namespace:  Interactive
File:       /usr/local/lib/python2.7/dist-

In [63]:
h5py.File.attrs

In [64]:
h5py.File.clear

<function collections.abc.MutableMapping.clear(self)>

In [65]:
h5py.File.close

<function h5py._hl.files.File.close(self)>

In [66]:
h5py.File.close?

Tiempo y Optimización

In [67]:
from timeit import timeit

In [68]:
import time


In [69]:
timeit('time.sleep(0.1)', number=5)

0.5019593280012487

In [70]:
%timeit time.sleep(0.1)

100 ms ± 45.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


packages/h5py/_hl/files.py
Definition: h5py.File.close(self)
Docstring:  Close the file.  All open objects become invalid
NOTA
Por defecto, IPython guardará la salida de sus declaraciones en variables especiales ocultas. Esto generalmente está bien, pero puede ser sorprendente si se cuelga de un objeto HDF5 que creía descartado, o de una gran matriz que consume memoria. Puede desactivar esto estableciendo el valor de configuración de IPython cache_sizeen 0. Consulte los documentos en http://ipython.org para obtener más información.

Tiempo y Optimización
Para las pruebas de rendimiento, utilizaremos el timeitmódulo que se incluye con Python. Los ejemplos que utilizan timeitsupondrán la siguiente importación:

>>> from timeit import timeit
La timeitfunción toma un comando (cadena o invocable) para ejecutarse, y se debe ejecutar un número opcional de veces. Luego imprime el tiempo total empleado ejecutando el comando. Por ejemplo, si ejecutamos la función "esperar" time.sleep cinco veces:

>>> import time
>>> timeit('time.sleep(0.1)', number=5)
0.49967818316434887
Si está utilizando IPython, existe una función "mágica" incorporada similar %timeitque ejecuta la instrucción especificada varias veces e informa el tiempo de ejecución más bajo:

>>> %timeit time.sleep(0.1)
10 loops, best of 3: 100 ms per loop
Seguiremos con la timeitfunción regular en este libro, en parte porque es proporcionada por la biblioteca estándar de Python.

Dado que las personas que usan HDF5 generalmente tratar con grandes conjuntos de datos, el rendimiento siempre es una preocupación. Pero notará que las discusiones de optimización y evaluación comparativa en este libro no entran en gran detalle sobre cosas como los éxitos de caché, las tasas de conversión de datos, etc. El diseño del paquete h5py, que utiliza este libro, deja casi todo eso 

en HDF5. De esta manera, se beneficiará de los cientos de años de trabajo que se dedicaron a sintonizar HDF5 para proporcionar el mayor rendimiento posible.

Como una aplicación Constructor, lo mejor que puede hacer para el rendimiento es utilizar la API de una manera sensata y dejar que HDF5 haga su trabajo. Aquí hay algunas sugerencias:

No optimices nada a menos que hayaUn problema de rendimiento demostrado. Luego, aísle cuidadosamente las partes del código que se comportan mal antes de cambiar cualquier cosa.
Comience con un código simple y directo que aproveche las características de la API. Por ejemplo, para iterar sobre todos los objetos en un archivo, use el Visitantecaracterística de HDF5 (consulte Iteración multinivel con el patrón de visitante ) en lugar de combinar su propio enfoque.
Realice mejoras "algorítmicas" primero. Por ejemplo, al escribir en un conjunto de datos (consulte el Capítulo 3 ), escriba datos en bloques de tamaño razonable en lugar de punto por punto. Esto permite que HDF5 use el sistema de archivos de manera inteligente.
Asegúrate de estar usando el tipos de datos correctos. Por ejemplo, si está ejecutando un programa intensivo en cómputo que carga datos de punto flotante de un archivo, asegúrese de no estar usando accidentalmente flotadores de doble precisión en un cálculo donde la precisión simple lo haría.
Finalmente, no dude en pedir ayuda en las listas de correo h5py o NumPy / Scipy, Stack Overflow u otros sitios de la comunidad. Muchas personas están usando NumPy y HDF5 en estos días, y muchos problemas de rendimiento tienen soluciones conocidas. La comunidad de Python es muy acogedora.


en HDF5. De esta manera, se beneficiará de los cientos de años de trabajo que se dedicaron a sintonizar HDF5 para proporcionar el mayor rendimiento posible.

Como una aplicación Constructor, lo mejor que puede hacer para el rendimiento es utilizar la API de una manera sensata y dejar que HDF5 haga su trabajo. Aquí hay algunas sugerencias:

No optimices nada a menos que hayaUn problema de rendimiento demostrado. Luego, aísle cuidadosamente las partes del código que se comportan mal antes de cambiar cualquier cosa.
Comience con un código simple y directo que aproveche las características de la API. Por ejemplo, para iterar sobre todos los objetos en un archivo, use el Visitantecaracterística de HDF5 (consulte Iteración multinivel con el patrón de visitante ) en lugar de combinar su propio enfoque.
Realice mejoras "algorítmicas" primero. Por ejemplo, al escribir en un conjunto de datos (consulte el Capítulo 3 ), escriba datos en bloques de tamaño razonable en lugar de punto por punto. Esto permite que HDF5 use el sistema de archivos de manera inteligente.
Asegúrate de estar usando el tipos de datos correctos. Por ejemplo, si está ejecutando un programa intensivo en cómputo que carga datos de punto flotante de un archivo, asegúrese de no estar usando accidentalmente flotadores de doble precisión en un cálculo donde la precisión simple lo haría.
Finalmente, no dude en pedir ayuda en las listas de correo h5py o NumPy / Scipy, Stack Overflow u otros sitios de la comunidad. Muchas personas están usando NumPy y HDF5 en estos días, y muchos problemas de rendimiento tienen soluciones conocidas. La comunidad de Python es muy acogedora.


View
La Figura 2-2 muestra el contenido de un archivo HDF5 con múltiples grupos en el panel izquierdo. Un grupo (llamado "1") está abierto, mostrando los conjuntos de datos que contiene; asimismo, se abre un conjunto de datos, con su contenido mostrado en la vista de cuadrícula a la derecha.

HDFView también le permite inspeccionar atributos de conjuntos de datos y grupos, y es compatible con casi todos los tipos de datos que HDF5 admite, con la excepción de ciertas estructuras de longitud variable.

ViTables
La Figura 2-3 muestra el mismo archivo HDF5 abierto en ViTables, otro navegador gráfico gratuito. Está optimizado para manejar archivos PyTables, aunque puede manejar archivos HDF5 genéricos perfectamente bien. Una de las principales ventajas de ViTables es que viene preinstalado con distribuciones de Python comoPythonXY, por lo que ya puede tenerlo.

ViTables
Figura 2-3. ViTables
Herramientas de línea de comando
Si está acostumbrado a la línea de comando, definitivamente vale la pena instalar la línea de comando HDF herramientas. Generalmente están disponibles a través de un administrador de paquetes; si no, puede obtenerlos en www.hdfgroup.org . Las versiones de Windows también están disponibles.

El programa que más usaremos en este libro se llama h5ls, que como su nombre indica enumera el contenido de un archivo. Aquí hay un ejemplo, en el que h5lsse aplica a un archivo que contiene un par de conjuntos de datos y un grupo:

$ h5ls demo.hdf5
conjunto de datos de matriz {10}
grupo grupo
conjunto de datos escalar {ESCALAR}
Podemos obtener información un poco más útil mediante el uso del combo de opciones -vlr, que imprime información extendida y también ingresa grupos de 

ntes de llegar a grupos y conjuntos de datos, comencemos explorando algunas de las capacidades de File objeto, que será su punto de entrada al mundo de HDF5.

Aquí está el programa más simple posible que usa HDF5:



f = h5py.File('name.hdf5')
f.close()

El Fileobjeto es tu punto de partida; tiene métodos que le permiten crear nuevos conjuntos de datos o grupos en el archivo, así como más propiedades peatonales como .filenamey .mode.

Hablando de .modeeso, los archivos HDF5 admiten el mismo tipo de modos de lectura / escritura que los archivos Python normales:
sto creará un nuevo archivo, pero fallará si ya existe un archivo con el mismo nombre. Por ejemplo, si está realizando un cálculo de larga duración y no quiere arriesgarse a sobrescribir su archivo de salida si el script se ejecuta dos veces, puede abrir el archivo en w-modo:



In [80]:

f = h5py.File('name.hdf5', 'w')   # New file overwriting any existing file


OSError: Unable to create file (file exists)

In [81]:
f = h5py.File('name.hdf5', 'r')   # Open read-only (must exist)


In [82]:
f = h5py.File('name.hdf5', 'r+')  # Open read-write (must exist)


In [83]:
f = h5py.File('name.hdf5', 'a')   # Open read-write (create if doesn't exist)fallara si existe el archivo


In [89]:

 f.close()

In [90]:
f = h5py.File('important_file.hdf5', 'w-')


OSError: Unable to create file (unable to open file: name = 'important_file.hdf5', errno = 17, error message = 'File exists', flags = 15, o_flags = c2)

Por cierto, eres libre utilizar nombres de archivo Unicode! Solo proporcione una cadena Unicode normal y funcionará de forma transparente, suponiendo que el sistema operativo subyacente sea compatible con la codificación UTF-8:


In [91]:
name = u'name_eta_\u03b7'


In [95]:
f = h5py.File(name, 'r')


In [97]:
print (f.filename)


name_eta_η


Usar como administrador de contexto
Una de las mejores características introducidas en Python 2.6 es la compatibilidad con los administradores de contexto . Estos son objetos con algunos métodos especiales llamados a la entrada y salida de un bloque de código, usando la withinstrucción. El ejemplo clásico es el fileobjeto Python incorpo

In [98]:
with open('somefile.txt', 'w') as f:
    f.write('Hello!')

El código anterior abre un nuevo fileobjeto, que está disponible en el bloque de código con el nombre f. Cuando el bloque sale, el archivo se cierra automáticamente (¡incluso si ocurre una excepción!).

El h5py.Fileobjeto admite exactamente este tipo de uso. Es una excelente manera de asegurarse de que el archivo siempre esté cerrado correctamente, sin envolver todo en try/ exceptblocks:



In [99]:
with h5py.File('name.hdf5', 'w') as f:
    print (f['missing_dataset'])


KeyError: "Unable to open object (object 'missing_dataset' doesn't exist)"

In [100]:
print (f)

<Closed HDF5 file>


Controladores de archivo
Los controladores de archivos se sientan entresistema de archivos y el mundo de nivel superior de grupos HDF5, conjuntos de datos y atributos. Se ocupan de la mecánica de mapear el "espacio de direcciones" HDF5 a una disposición de bytes en el disco. Por lo general, no tendrá que preocuparse por qué controlador está en uso, ya que el controlador predeterminado funciona bien para la mayoría de las aplicaciones.

Lo mejor de los controladores es que una vez que se abre el archivo, son totalmente transparentes. Simplemente usa la biblioteca HDF5 de forma normal y el controlador se encarga de la mecánica de almacenamiento.

Aquí hay algunos de los más interesantes, que pueden ser útiles para problemas inusuales.

conductor central
El coreconductoralmacena su archivo completamente en la memoria . Obviamente, hay un límite para la cantidad de datos que puede almacenar, pero la compensación es de lectura y escritura increíblemente rápida. Es una gran opción cuando desea la velocidad de acceso a la memoria, pero también quiere usar las estructuras HDF5. Para habilitar, establezca la driverpalabra clave en 'core':



In [103]:
f = h5py.File('name.hdf5','r', driver='core')


In [104]:
f = h5py.File('name.hdf5','r', driver='core', backing_store=True)


Por cierto, la backing_storepalabra clave también le indicará a HDF5 que cargue cualquier imagen existente del disco cuando abra el archivo. Entonces, si todo el archivo cabe en la memoria, debe leer y escribir la imagen solo una vez; cosas como lecturas y escrituras de conjuntos de datos, creación de atributos, etc., no tome ninguna E / S de disco.

conductor familiar
A veces es conveniente dividir un archivo en varias imágenes, todas las cuales comparten un cierto tamaño máximo. Esta característica se implementó originalmente para admitir sistemas de archivos que no podían manejar tamaños de archivo superiores a 2 GB

In [108]:
f = h5py.File('family.hdf5','r', driver='family', memb_size=1024**3)# Split the file into 1-GB chunks

OSError: Unable to open file (file names not unique)

El valor predeterminado para memb_sizees 2 31 -1, de acuerdo con los orígenes históricos del controlador.

controlador de ventana
Este conductor es el corazón de paralelo HDF5. Le permite acceder al mismo archivo desde múltiples procesos al mismo tiempo. Puede tener docenas o incluso cientos de procesos informáticos paralelos, todos los cuales comparten una vista coherente de un solo archivo en el disco.

Usar el mpiocontrolador correctamente puede ser complicado. El Capítulo 9 cubre tanto los detalles de este controlador como las mejores prácticas para usar HDF5 en un entorno paralelo.

El bloque de usuario
Una característica interesante de HDF5 es que los archivos pueden ser precedidos por datos arbitrarios del usuario. Cuando se abre un archivo, la biblioteca busca el encabezado HDF5 al comienzo del archivo, luego 512 bytes, luego 1024, y así sucesivamente en potencias de 2. Tal espacio al comienzo del archivo se llama "bloque de usuario ", Y puede almacenar los datos que desee allí.

Las únicas restricciones sonen el tamaño del bloque (potencias de 2, y al menos 512), y que no debe tener el archivo abierto en HDF5 al escribir en el bloque de usuario. Aquí hay un ejemplo:

In [109]:
f = h5py.File('userblock.hdf5', 'w', userblock_size=512)


In [110]:
f.userblock_size

512

In [111]:
f.close()

In [123]:
with open('userblock.hdf5', 'r+') as f:# Open as regular Python file
    f.write('a'*512)

Pasemos al primer objeto principal en el modelo de datos HDF5, uno que será familiar para los usuarios del tipo de matriz NumPy: conjuntos de datos.

Los conjuntos de datos son la característica central de HDF5. Puede pensar en ellos como matrices NumPy que viven en el disco. Cada conjunto de datosen HDF5 tiene un nombre, un tipo y una forma, y ​​admite acceso aleatorio. A diferencia del incorporado np.savey los amigos, no hay necesidad de leer y escribir toda la matriz como un bloque; puede usar la sintaxis estándar de NumPy para cortar y leer y escribir solo las partes que desee.



In [125]:
f = h5py.File('testfile.hdf5','r')

In [126]:
arr = np.ones((5,2))

In [127]:
f['my dataset'] = arr

In [128]:
dset = f['my dataset']

In [129]:
dset

<HDF5 dataset "my dataset": shape (5, 2), type "<f8">

Pusimos una matriz NumPy pero recuperamos algo más: una instancia de la clase h5py.Dataset. Este es un objeto "proxy" que le permite leer y escribir en el conjunto de datos HDF5 subyacente en el disco.

Tipo y forma
Vamos a explorarEl Datasetobjeto. Si está utilizando IPython, escriba dset.y presione Tab para ver los atributos del objeto; de lo contrario, hazlo dir(dset). Hay muchos, pero algunos se destacan

In [130]:
dset.dtype

dtype('<f8')

Cada conjunto de datos tiene untipo fijo que se define cuando se crea y nunca se puede cambiar. HDF5 tiene un mecanismo de tipo amplio y expresivo que puede 

manejar fácilmente los tipos NumPy incorporados, con pocas excepciones. Por esta razón, h5py siempre expresa el tipo de un conjunto de datos utilizando dtypeobjetos NumPy estándar .

Hay otro atributo familiar:


La forma de un conjunto de datos también se define cuando se crea, aunque, como veremos más adelante, se puede cambiar. Al igual que las matrices NumPy, los conjuntos de datos HDF5 pueden tener entre cero ejes (escalar, forma ()) y 32 ejes. Los ejes del conjunto de datos pueden tener hasta 2 63 -1 elementos de longitud.



In [131]:
dset.shape


(5, 2)

Leyendo y escribiendo
Los conjuntos de datos no serían tan útiles si no pudiéramos obtener los datos subyacentes. Primero, veamos qué sucede si solo leemos todo el conjunto de datos:



In [132]:
out = dset[...]#leer todo los datos

In [133]:
out

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])

In [134]:
type(out)

numpy.ndarray

Cortando en un Datasetobjeto devuelve una matriz NumPy. Tenga en cuenta lo que realmente sucede cuando hace esto: h5py traduce su selección de corte en una porción del conjunto de datos y hace que HDF5 lea los datos del disco. En otras palabras, ignorando el almacenamiento en caché, una operación de corte resulta en una lectura o escritura en el disco.

Intentemos actualizar solo una parte del conjunto de datos:



In [135]:
dset[1:4,1] = 2.0

In [136]:
dset[...]

array([[1., 1.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 1.]])

Debido a que los Datasetobjetos son muy similares a las matrices NumPy, puede tener la tentación de mezclarlos con el código computacional. Esto puede funcionar por un tiempo, pero generalmente causa problemas de rendimiento ya que los datos están en el disco en lugar de en la memoria.

Crear conjuntos de datos vacíos
No necesitas tener una matriz NumPy lista para crear un conjunto de datos. El método create_dataseten nuestro Fileobjeto puede crear conjuntos de datos vacíos a partir de una forma y tipo, o incluso solo una forma (en cuyo caso el tipo será np.float32flotante nativo de precisión simple):



In [137]:
dset = f.create_dataset('test1', (10, 10))

In [138]:
dset

<HDF5 dataset "test1": shape (10, 10), type "<f4">

In [139]:
dset = f.create_dataset('test2', (10, 10), dtype=np.complex64)

In [140]:
dset

<HDF5 dataset "test2": shape (10, 10), type "<c8">

HDF5 es lo suficientemente inteligente como para asignar solo tanto espacio en el disco como lo que realmente necesita para almacenar los datos que escribe. Aquí hay un ejemplo: suponga que desea crear un conjunto de datos 1D que pueda contener 4 gigabytes de muestras de datos de un experimento de larga duración:



In [141]:
dset = f.create_dataset('big dataset', (1024**3,), dtype=np.float32)


In [142]:
dset

<HDF5 dataset "big dataset": shape (1073741824,), type "<f4">

Ahora escríbele algunos datos. Para ser justos, también le pedimos a HDF5 que vacíe sus buffers y que realmente escriba en el disco:

In [143]:
dset[0:1024] = np.arange(1024)

In [144]:
 f.flush()

In [145]:
dset

<HDF5 dataset "big dataset": shape (1073741824,), type "<f4">

In [146]:
ls -lh testfile.hdf5

-rw-r--r-- 1 f f 4,1G ene  4 17:44 testfile.hdf5


In [147]:
ls

01_station_15.ipynb                   img/
01-Structuring-Datasets.ipynb         important_file.hdf5
02-Datatypes-in-HDF5.ipynb            mandel.py
03-Chunking.ipynb                     name_eta_η
04-Using-Compression.ipynb            name.hdf5
05-Queries-in-PyTables.ipynb          NumPy-Ejercicios-Serie3.ipynb
06-Indexed-Queries-in-PyTables.ipynb  solutions/
07-Expressions.ipynb                  somefile.txt
08-Integration-With-Pandas.ipynb      testfile.hdf5
09-Low-Level-API.ipynb                userblock.hdf5
10-Parallel-HDF5.ipynb                weather.hdf5
datasets/                             weather.npz
demo_mpi.py


Cuando se trata de tipos, unos pocos segundos de reflexión pueden ahorrarle mucho espacio en disco y también reducir su tiempo de E / S. El create_datasetmetodo puede aceptar casi cualquier tipo de tipo NumPy para el conjunto de datos subyacente y, lo que es más importante, no tiene que coincidir exactamente con el tipo de datos que luego escriba en el conjunto de datos.

Aquí hay un ejemplo: un uso común para HDF5 es almacenar datos numéricos de coma flotante, por ejemplo, series de tiempo de digitalizadores, precios de acciones, simulaciones por computadora, en cualquier lugar donde sea necesario representar números del "mundo real" que no sean enteros.

A menudo, para mantener la precisión de Para cálculos altos, se utilizarán números de doble precisión de 8 bytes en la memoria (tipo NumPy float64), para minimizar los efectos del error de redondeo. Sin embargo, es una práctica común almacenar estos puntos de datos en el disco como números de 4 bytes de precisión simple ( float32), ahorrando un factor de 2 en el tamaño del archivo.

Supongamos que tenemos una matriz NumPy llamada bigdata:



In [174]:
 bigdata = np.ones((100,1000))

In [175]:
bigdata.dtype


dtype('float64')

In [176]:
bigdata.shape

(100, 1000)

Podríamos almacenar esto en un archivo por simple asignación, resultando en un conjunto de datos de doble precisión:



In [177]:
with h5py.File('big1.hdf5','w') as f1:
    f1['big'] = bigdata


OSError: Unable to create file (unable to truncate a file which is already open)

In [178]:
ls -lh big1.hdf5

-rw-r--r-- 1 f f 784K ene  4 17:55 big1.hdf5


Tenga en cuenta que cualquiera que elija, sus datos surgirán del archivo en ese formato:

In [179]:
f1 = h5py.File('big.hdf5')

<ipython-input-179-b367d48c77c4>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f1 = h5py.File('big.hdf5')


In [180]:
f2 = h5py.File('big2.hdf5')

<ipython-input-180-f3291c55ae27>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f2 = h5py.File('big2.hdf5')
